### 河合塾

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re

import pandas as pd

In [2]:
DATA = []

driver = webdriver.PhantomJS()
driver.get('http://www.kawai-juku.ac.jp/trial-exam/zento/lnp/2nd/') #  ここを変える
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit

for kaijo in soup.find('div', class_='tableBlockA01 mt00').findAll(lambda tag: tag.name=='a' and tag.find('img', alt='試験会場実施日')):
    driver.get('http://www.kawai-juku.ac.jp' + kaijo.attrs['href'])
    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    print(soup2.title.get_text().split(' | ')[0:2])
    info = soup2.title.get_text().split(' | ')[0:2]
    driver.get(soup2.iframe.attrs['src'])
    tags = driver.find_elements_by_xpath("//a[@href='javascript:;']")
    
    for i in range(len(tags)):
        driver.get('http://www.kawai-juku.ac.jp' + kaijo.attrs['href'])
        driver.get(BeautifulSoup(driver.page_source, 'html.parser').iframe.attrs['src'])
        driver.find_elements_by_xpath("//a[@href='javascript:;']")[i].click()
        time.sleep(0.5)
        soup3 = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit
        # print(soup3.find('td', bgcolor="#ffffff").get_text().replace('\n', '').replace(' ','').replace('\t', '').replace('　', ''))
        d = soup3.find('td', bgcolor="#ffffff").get_text().replace('\n', '').replace(' ','').replace('\t', '').replace('　', '')
        
        address = []
        for f in soup3.tbody.tbody.td.findAll('table')[5].td.findAll('font', {'color':None}):
            # print(f.get_text().replace('　','').replace('\xa0','').split('\n'))
            address.append(f.get_text().replace('　','').replace('\xa0','').replace('\n', ''))
        # print(address)
        
        DATA.append([info[0].split('(')[0], info[1], d, address[0], address[1]])

['全統高2記述模試（記述・論述式） 試験会場実施日', '高2生']
['全統マーク高2模試 試験会場実施日', '高2生']


In [3]:
dat = pd.DataFrame(DATA, columns=['moshi', 'grade', 'date', 'place', 'address'])
dat.head()

,moshi,grade,date,place,address
0,全統高2記述模試（記述・論述式） 試験会場実施日,高2生,2018/02/04,河合塾札幌校,札幌市北区北９条西３丁目３番地
1,全統高2記述模試（記述・論述式） 試験会場実施日,高2生,2018/01/28,東北大川内北キャンパス講義棟,仙台市青葉区川内４１
2,全統高2記述模試（記述・論述式） 試験会場実施日,高2生,2018/02/04,茨城大学水戸ＣＰ,水戸市文京２丁目１番１号
3,全統高2記述模試（記述・論述式） 試験会場実施日,高2生,2018/02/04,宇都宮大学峰キャンパス,宇都宮市峰町３５０
4,全統高2記述模試（記述・論述式） 試験会場実施日,高2生,2018/02/04,育英メディカル専門学校,前橋市古市町１－３５－６


In [4]:
dat.to_csv('res/kawai.csv')